In [14]:
import time
import pickle
from selenium import webdriver
from urllib.parse import urlparse
from bs4 import BeautifulSoup
from gsheet import authenticate
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By

In [15]:
# Open Browser
browser = webdriver.Firefox()
# chrome = webdriver.Chrome()

# Constants
google = "https://google.com"
first_template_doc_url = "https://docs.google.com/document/d/1Fw-QKu1pSZdkIMPGuhPMTlgsoCum4lDttfNmvUOaeec/edit?usp=sharing"
outlook_office_url = "https://outlook.office.com/"
outlook_live_url = "https://outlook.live.com/"
fortest = "https://docs.google.com/spreadsheets/d/1UGhTjkslKEjoYA0mDe5l8GtS80hS39IsTe0YTE5s0a8/edit#gid=0"
gsheet_creds = "creds/s_a_creds.json"
tab_name = "Sheet2"
action = webdriver.ActionChains(browser)

In [16]:
# Functions

# Load Old Cookies (If exists)
def load_cookies(driver, location, url=None):
    cookies = pickle.load(open(location, "rb"))
    driver.delete_all_cookies()
    url = "https://www.google.com" if url is None else url
    driver.get(url)
    for cookie in cookies:
        driver.add_cookie(cookie)


# Credentials for used account
def get_email_creds(file_name):
    fl = open(file_name).readlines()
    email = fl[0]
    password = fl[1]
    return [email, password]


# Go to google docs url and copy the template
def grab_template(template):
    browser.get(template)
    time.sleep(5)
    action.key_down(Keys.CONTROL).send_keys("a").key_up(Keys.CONTROL).perform()
    time.sleep(1)
    action.key_down(Keys.CONTROL).send_keys("c").key_up(Keys.CONTROL).perform()

    
# Login to Normal Outlook Live Account
def login_to_outlook_live(email, password):
    browser.get("https://login.live.com")
    time.sleep(3)
    mail_id = browser.find_element_by_name("loginfmt")
    mail_id.send_keys(email)
    mail_id.send_keys(Keys.RETURN)
    time.sleep(2)
    passwd = browser.find_element_by_name("passwd")
    passwd.send_keys(password)
    passwd.submit()
    time.sleep(3)
    browser.get("https://outlook.live.com")


# Login to Professional Outlook Office account
def login_to_outlook_office(email, password):
    browser.get("https://outlook.office.com")
    time.sleep(3)
    mail_id = browser.find_element_by_name("loginfmt")
    mail_id.send_keys(email)
    mail_id.send_keys(Keys.RETURN)
    time.sleep(2)
    passwd = browser.find_element_by_name("passwd")
    passwd.send_keys(password)
    passwd.submit()
    time.sleep(3)
    browser.get("https://outlook.live.com")
    

# Connect To The Google Sheets API
def connect_to_gsheet(credentials, url, tab_name):
    client = authenticate(credentials)
    workbook = client.open_by_url(url)
    selected_tab = workbook.worksheet(tab_name)
    return selected_tab
    
    
# Send An Email
def send_email_from_sarah(email, first_name, company_name, duration):
    new_msg = browser.find_elements_by_class_name("_33rLSYbzxvhXjgYTwfjWQI")
    new_msg[0].click()
    time.sleep(duration)
    to = browser.find_elements_by_class_name("pickerInput_cf204fb8")
    to[0].send_keys(email)
    time.sleep(duration)
    to[0].send_keys(Keys.ENTER)
    time.sleep(2)
    subject = browser.find_elements_by_xpath("//input[@placeholder='Add a subject']")
    subject[0].click()
    subject[0].send_keys(f"Private, online data science training exclusively for {company_name}")
    time.sleep(duration)
    msg_body = browser.find_elements_by_xpath("//div[@aria-label='Message body']")
    msg_body[0].click()
    msg_body[0].send_keys(f"Hi {first_name},\n\n")
    time.sleep(2)
    msg_body[0].send_keys(Keys.CONTROL + "v")
    time.sleep(duration)
    msg_body[0].send_keys(Keys.BACKSPACE)
    time.sleep(2)
    confirmation = input(f"Before sending please check that,\nThe Person's name is {first_name},\nHis mail address is {email}\nAnd he works in {company_name}\nWrite s and Press ENTER to SEND the mail to {first_name}")
    if confirmation.lower() == "s":
        msg_body[0].send_keys(Keys.CONTROL + Keys.ENTER)
        print(f"Email sent to {first_name}\n")
    else:
        print(f"The mail is saved to the Draft Folder.\n")


# use outlook live account
email_creds = get_email_creds("creds/outlook_neela.txt")
login_to_outlook_live(email_creds[0], email_creds[1])
time.sleep(2)
pickle.dump(browser.get_cookies(), open("cookies/outlook_neela.txt", "wb"))

# Use Outlook Office Account
# try:
#     load_cookies(browser, "cookies/outlook_sarah.txt" url=outlook_office_url)
# except:
#     email_creds = get_email_creds("creds/outlook_sarah.txt")
#     login_to_outlook_office(email_creds[0], email_creds[1])
#     time.sleep(2)
#     pickle.dump(browser.get_cookies(), open("cookies/outlook_sarah.txt", "wb"))


selected_tab = connect_to_gsheet(gsheet_creds, fortest, tab_name)


In [17]:
grab_template(first_template_doc_url)
browser.get(outlook_live_url)
# Select rows and send mail to them
print("Google Sheet Row Data:")
starting_row = int(input("Enter the Starting Row Number: "))
finishing_row = int(input("Enter the Finishing Row Number: "))
companies = selected_tab.get(f"A{starting_row}:A{finishing_row}")
first_names = selected_tab.get(f"E{starting_row}:E{finishing_row}")
emails = selected_tab.get(f"H{starting_row}:H{finishing_row}")
total_rows = finishing_row - starting_row + 1
print(f"You have selected {total_rows} rows.")
# Loop
for i in range(total_rows):
    email = emails[i][0]
    company_name = companies[i][0]
    first_name = first_names[i][0]
    send_email_from_sarah(email, first_name, company_name, 5)
    if i == total_rows - 1:
        new_msg = browser.find_elements_by_class_name("_33rLSYbzxvhXjgYTwfjWQI")
        new_msg[0].click()
        print(f"\n{total_rows} emails sent.")

Google Sheet Row Data:
Enter the Starting Row Number: 915
Enter the Finishing Row Number: 920
You have selected 6 rows.
Before sending please check that,
The Person's name is Aaron,
His mail address is mahmudur.rahman99@gmail.com
And he works in The Trade Desk
Write s and Press ENTER to SEND the mail to Aarons
Email sent to Aaron

Before sending please check that,
The Person's name is Alexa,
His mail address is shovon@marsdecoder.com
And he works in The Trade Desk
Write s and Press ENTER to SEND the mail to Alexas
Email sent to Alexa

Before sending please check that,
The Person's name is Ally,
His mail address is mahmudur.rahman99@outlook.com
And he works in The Trade Desk
Write s and Press ENTER to SEND the mail to Allys
Email sent to Ally

Before sending please check that,
The Person's name is Amanda,
His mail address is 8shovon8@gmail.com
And he works in The Trade Desk
Write s and Press ENTER to SEND the mail to Amandas
Email sent to Amanda

Before sending please check that,
The Pe